# ETL Pipeline Preparation
Follow the instructions below to help you create your ETL pipeline.
### 1. Import libraries and load datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [1]:
# import libraries
import pandas as pd
import nltk
nltk.download('punkt')
from sqlalchemy import create_engine

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\markf\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# load messages dataset
def load_messages(file_name):
    
    return pd.read_csv(file_name, index_col='id')

In [3]:
# load categories dataset
def load_categories(file_name):
    
    return pd.read_csv(file_name, index_col='id')

### 2. Split `categories` into separate category columns.
- Split the values in the `categories` column on the `;` character so that each value becomes a separate column. You'll find [this method](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.Series.str.split.html) very helpful! Make sure to set `expand=True`.
- Use the first row of categories dataframe to create column names for the categories data.
- Rename columns of `categories` with new column names.

#### 2.1. Convert category values to just numbers 0 or 1.
- Iterate through the category columns in df to keep only the last character of each string (the 1 or 0). For example, `related-0` becomes `0`, `related-1` becomes `1`. Convert the string to a numeric value.
- You can perform [normal string actions on Pandas Series](https://pandas.pydata.org/pandas-docs/stable/text.html#indexing-with-str), like indexing, by including `.str` after the Series. You may need to first convert the Series to be of type string, which you can do with `astype(str)`.

In [4]:
# create a dataframe of the 36 individual category columns
def expand_categories(df, column_name, col_sep=';', value_sep='-'):

    # create a dataframe of the 36 individual category columns
    categories = df_cat[column_name].str.split(col_sep, expand=True)
    
    # select the first row of the categories dataframe
    row = categories.iloc[0, :]
    
    # use this row to extract a list of new column names for categories.
    categories.columns = row.str.split(value_sep, expand=True)[0]
    
    for column in categories:
        # set each value to be the last character of the string
        categories[column] = categories[column].str.split(value_sep, expand=True)[1]

        # convert column from string to numeric
        categories[column] = pd.to_numeric(categories[column])
    
    return categories

### 3. Merge datasets.
- Merge the messages and categories datasets using the common id
- Assign this combined dataset to `df`, which will be cleaned in the following steps

In [5]:
# merge datasets
def merge_msg_cat(df_msg, df_cat):

    df_merge = df_msg.merge(df_cat, left_index=True, right_index=True, how='inner')
    df_merge = df_merge.reset_index() # After merge on id, use a reliable unique index
    return df_merge

### 4. Clean the data
- Check how many duplicates are in this dataset.
- Drop columns not used in the machine learning pipeline.
- Drop repeated rows (keeping the first occurrence)
- Drop rows with same values in the independent var (remove all).
- Replace incoherent values in the dataset.

In [6]:
def clean(df, drop_columns, independent_vars, replacements):
    
    # Drop columns that shouldn't be read in the machine learning pipeline.
    print("Dropping columns:", drop_columns)
    df_clean = df.drop(columns=drop_columns)
    
    # Drop identical rows
    df_clean = df_clean.drop_duplicates()
    
    # Drop rows with same predictor values with different predicted values
    df_clean = df_clean.drop_duplicates(subset=independent_vars, keep=False)
    
    df_clean = df_clean.dropna()
    
    # Replace values in columns
    for col, rep in replacements:
        df_clean[col] = df_clean[col].replace(rep[0], rep[1])
    return df_clean

### 5. Save the clean dataset into an sqlite database.

Connect to the database and save the data. The data will be stored into a table. Keep this table name to read the data and create tha Machine Learning Pipeline.

In [7]:
def save(df, database, table_name):
    engine = create_engine('sqlite:///'+database)
    df.to_sql(table_name, engine, if_exists='replace')

### 6. Call all functions

In [17]:
df_msg = load_messages(file_name='disaster_messages.csv')
df_cat = load_categories(file_name='disaster_categories.csv')

df_cat_exp = expand_categories(df_cat, column_name='categories')

df_merge = merge_msg_cat(df_msg, df_cat_exp)

df_clean = clean(df_merge,
                 drop_columns=['genre', 'original', 'child_alone'],# Column child_alone is filled with zeros
                 independent_vars=['message'], # The predictor variable
                 replacements=[('related', (2,1))])# Replace 2 by 1 in the column 'related'

save(df_clean, database='DisasterResponse.db', table_name='Messages')

Dropping columns: ['genre', 'original', 'child_alone']
